# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv
import json
import requests

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import LocalTarget, ComputeTargetException

import numpy as np
import pandas as pd

from azureml.train.automl import utilities
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

from azureml.core import Model
from azureml.core import Webservice

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


## Dataset

### Overview
Data were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features from images.

It is a classification problem to find out whether the bank note is genuine or forged.

The above note and dataset have been collected from [UCI](https://archive.ics.uci.edu/ml/datasets/banknote+authentication).


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

#Getting dataset that is registered onto workspace
dataset = Dataset.get_by_name(ws, name='Dataset_bank_note_auth')
df = dataset.to_pandas_dataframe()

print("Shape of data: ", str(df.shape))

label = 'Class'

In [ ]:
#View dataset
df.head(10)

In [ ]:
# Name of the experiment
experiment_name = 'auto_ml_capstone_project'
experiment=Experiment(ws, experiment_name)

experiment

# Create compute instance

In [ ]:
# Choose a name for your CPU cluster
amlcompute_name = "aml-compute"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=6)
    aml_compute = ComputeTarget.create(ws, amlcompute_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The following are reasons.

* experiment_timeout_minutes = 15 (This depicts total time to be taken by all iterations before it terminates.)
* max_concurrent_iterations = 5 (It represents maximum number of parallel execution.)
* enable_early_stopping = True (It helps terminate execution if score is not improving in the short term.)
* task = classification (This is type of execution to solve the problem.)
* primary_metric = accuracy (Accuracy being chosen which will help choose the best model selection.)
* training_data = train_data (This is train data i.e. bank note authentication.)
* label_column_name = label (The name of the label column.)
* n_cross_validations = 5 (Number of cross validation to be performed.)
* compute_target = aml_compute (Experiment to be run on compute.)

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name=label,
    n_cross_validations=5,
    compute_target=aml_compute,
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = automl_run.get_output()
print(best_run, "\n")
print(best_run.get_metrics(), "\n")
print(best_model._final_estimator)


In [ ]:
best_run.get_file_names()

In [ ]:
#TODO: Save the best model
import joblib

best_run.download_file(name="outputs/model.pkl", output_file_path="./automl_model.pkl")

# Register
model = best_run.register_model(
    model_name='automl_model', 
    model_path='outputs/model.pkl',
)
model

In [ ]:
joblib.load("./automl_model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")
conda_dep = CondaDependencies()


# # Define the packages needed by the model and scripts
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
# # You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("azureml-core")
conda_dep.add_pip_package("joblib")
conda_dep.add_pip_package("azureml-train-automl")


# # Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

myenv.register(workspace=ws)

myenv = Environment.get(workspace=ws, name="myenv")



In [ ]:
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=True) 

In [ ]:
service = Model.deploy(workspace=ws, 
                       name='automl-model-service', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output = True)

In [ ]:
data = {
  "data": [
    {
                  "Variance": -3.8952,
                  "Skewness": 3.8157,
                  "Curtosis": -0.31304,
                  "Entropy": -3.8194
    }
  ]
}

print (data)
input_data = json.dumps(data)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json
import numpy as np

output = service.run(input_data)

print(f"Predicted: {output}")

In [ ]:
#Status of endpoint
print(f'\nservice state: {service.state}\n')

#Scoring URI
print(f'scoring URI: \n{service.scoring_uri}\n')

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

# Clean up


In [ ]:
service.delete()
aml_compute.delete()